In [3]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from yuzhi.model import MiniMonkeyChatModel, InternLM2Tokenizer
# from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images, target_aspect_ratio


def dynamic_preprocess2(image, min_num=1, max_num=12, prior_aspect_ratio=None, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    new_target_ratios = []
    for i in target_ratios:
        if prior_aspect_ratio[0]%i[0] or prior_aspect_ratio[1]%i[1]:
            new_target_ratios.append(i)
        else:
            continue
    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, new_target_ratios, orig_width, orig_height, image_size)
    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, min_num=1, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images, target_aspect_ratio = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, min_num=min_num, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values, target_aspect_ratio

def load_image2(image_file, input_size=448, min_num=1, max_num=12, target_aspect_ratio=None):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess2(image, image_size=input_size, use_thumbnail=True, min_num=min_num, max_num=max_num, prior_aspect_ratio=target_aspect_ratio)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

check_point = '/home/zhangjingbo/Workspace/Mini-Monkey'
# model = AutoModel.from_pretrained(
#     path,
#     torch_dtype=torch.bfloat16).eval().cuda()
# tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

model = MiniMonkeyChatModel.from_pretrained(
    check_point,
    torch_dtype=torch.bfloat16).eval().cuda()
tokenizer = InternLM2Tokenizer.from_pretrained(check_point)

# use your own image
#image_path = '/home/username/images/movie_cap.png'
image_path = '/home/username/images/fruits_and_drink.jpeg'
# set the max number of tiles in `max_num`
pixel_values, target_aspect_ratio = load_image(image_path, min_num=4, max_num=12)
pixel_values = pixel_values.to(torch.bfloat16).cuda()
pixel_values2 = load_image2(image_path, min_num=3, max_num=7, target_aspect_ratio=target_aspect_ratio)
pixel_values2 = pixel_values2.to(torch.bfloat16).cuda()
pixel_values = torch.cat([pixel_values[:-1], pixel_values2], 0)

generation_config = dict(do_sample=False, max_new_tokens=512)

#question = "Read the all text in the image."
#question = "Describe the image."
question = "描述这张图片。"
response, history = model.chat(tokenizer, pixel_values, target_aspect_ratio, question, generation_config, history=None, return_history=True)
print(f'User: {question} Assistant: {response}')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


User: 描述这张图片。 Assistant: 这张图片展示了一瓶饮料和一个水果拼盘。

1. **饮料瓶**：
   - 瓶子是透明的，里面装满了粉红色的饮料。
   - 瓶子的标签上有一些文字和图案，包括水果和饮料的图案。
   - 瓶子的顶部有一个红色的瓶盖，上面有白色的文字。

2. **水果拼盘**：
   - 拼盘中有多种水果，包括：
     - 一个红色的苹果
     - 一个黄色的苹果
     - 一个绿色的苹果
     - 一个红色的草莓
     - 一个黄色的柠檬
     - 一个黄色的梨
     - 一个黄色的桃子
     - 一个黄色的芒果
     - 一个黄色的葡萄
     - 一个红色的葡萄
     - 一个绿色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
     - 一个黄色的葡萄
